<a href="https://colab.research.google.com/github/ashishtrivedi16/major_project_stuff/blob/master/fingerprint_inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# GPU details
!nvidia-smi

In [0]:
# CPU details
!cat /proc/cpuinfo

In [0]:
'''
Mount your gdrive account by running this cell and place your database files
in it for easy access.
'''
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os # for basic operations like folder creation, directory validation etc
import glob # finds file according to wildcard given
import shutil # for moving files
import matplotlib.pyplot as plt # for plotting graphs and viewing images

In [0]:
'''
This function organises fingerprint of a person to his respective folder
No need to run this cell as i've already organised the images into their
respective folders
'''

src = "./drive/My Drive/dbs/DB1_B/dataset"
des = "./drive/My Drive/dbs/DB1_B/dataset"
def organise_files(path=src):
    for i in range(101, 111):
        if not os.path.isdir(src + str(i)):
            os.makedirs(src + str(i))
        for name in glob.glob(src + str(i) + "_*.tif"):
            shutil.move(name, des + str(i) )

organise_files()            

In [0]:
'''
Visualise file directory structure
'''

startpath = './drive/My Drive/dbs/DB1_B'

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

list_files(startpath)

In [64]:
'''
function to divide the dataset into train and test folders for 
ImageDataGenerator.flow_from_directory() function

train set -> 80% (6 fingerprints per person)
validation set -> 20% (2 fingerprints per person)
'''

startpath = './drive/My Drive/dbs/DB1_B/'
train_split = 0.80
validation_split = 0.20

def train_validation_split(startpath, train=train_split, validation=validation_split):
    # first create same sub-diretory structure in both train and validation folders
    dir_list  = [x[1] for x in os.walk(startpath + 'dataset')][0]
    for dir in dir_list:
        if not os.path.isdir(startpath + 'train/' + dir):
            os.mkdir(startpath + 'train/' + dir)
        if not os.path.isdir(startpath + 'validation/' + dir):
            os.mkdir(startpath + 'validation/' + dir)

    # randomly select images based on train validation slipt and move them
    # to their respective folders
    filename_list = [x[0] for x in os.walk(startpath + 'dataset')][1:]
    print(filename_list)


train_validation_split(startpath)


['./drive/My Drive/dbs/DB1_B/dataset/110', './drive/My Drive/dbs/DB1_B/dataset/105', './drive/My Drive/dbs/DB1_B/dataset/103', './drive/My Drive/dbs/DB1_B/dataset/102', './drive/My Drive/dbs/DB1_B/dataset/104', './drive/My Drive/dbs/DB1_B/dataset/108', './drive/My Drive/dbs/DB1_B/dataset/106', './drive/My Drive/dbs/DB1_B/dataset/107', './drive/My Drive/dbs/DB1_B/dataset/101', './drive/My Drive/dbs/DB1_B/dataset/109']


In [0]:
'''
-> Generator class for applying image transformations
-> Can be used for image augmentation purposes if need be
-> Currently just normalises the image
-> tensorflow does not support importing and processing images
    so use this or use PIL(python image library)
'''
from keras.preprocessing.image import ImageDataGenerator 

# Image dimension data
'''
    Sensor Type	    Image Size	            Set A	Set B	Resolution
DB1	Optical Sensor	640x480 (307 Kpixels)	100x8	10x8	500 dpi
DB2	Optical Sensor	328x364 (119 Kpixels)	100x8	10x8	500 dpi
DB3	Thermal sweeping
    Sensor	        300x480 (144 Kpixels)	100x8	10x8	512 dpi
DB4	SFinGe v3.0	    288x384 (108 Kpixels)	100x8	10x8	about 500 dpi
'''

img_width, img_height = 150, 150
train_data_dir = './drive/My Drive/dbs/DB1_B/train'
validation_data_dir = './drive/My Drive/dbs/DB1_B/validation'
nb_train_samples = 2000
nb_validation_samples = 800
batch_size = 10
epochs = 150

if K.image_data_format() == 'channels_last':
    input_tensor = Input(shape=(img_width, img_height, 3))
else:
    input_tensor = Input(shape=(3, img_width, img_height))


train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './drive/My Drive/dbs/DB1_B/',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800)

from keras.models import Model
from keras import backend as K
from keras.layers import Input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3

# imports the inceptionv3 pretained model
inception_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inception_model.input, outputs=predictions)

# freeze all convolutional InceptionV3 layers
for layer in inception_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.summary()

model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)